# Forest Cover Type with SageMaker Experiments - Introduction

This series of notebooks demonstrates techniques for tabular data ML in SageMaker, on the popular **"Forest Cover Type"** multiclass classification task.

**This notebook** handles initial loading of the data and some basic transformations: After which you'll be ready to run the follow-on notebooks to train and deploy predictive models.

## About the task & Acknowledgements

The Forest Cover Type dataset is copyright Jock A. Blackard and Colorado State University, and made available to us via the [**UCI Machine Learning Repository page**](https://archive.ics.uci.edu/ml/datasets/covertype).

The task is to predict, for each of 581012 patches of forest in northern Colorado, which one of 7 types types of tree cover dominate from 54 numerical input variables.

See [**Forest Cover Type Classification Study**](https://rstudio-pubs-static.s3.amazonaws.com/160297_f7bcb8d140b74bd19b758eb328344908.html) (Thomas Kolasa and Aravind Kolumum Raja) for a really nicely-presented review of the problem with traditional data science methods and interactive graphics!

## Setting up our environment

Note that [sagemaker-experiments](https://github.com/aws/sagemaker-experiments) ([docs](https://sagemaker-experiments.readthedocs.io/en/latest/)) is a **separate library** from the [SageMaker SDK](https://sagemaker.readthedocs.io/en/stable/).

We'll start with some installs to make these notebooks more consistent across different kernel environments:

In [ ]:
# sagemaker-experiments SDK is not installed on SageMaker notebooks by default.
# tqdm is available in some kernels but not others, between Studio and Notebook Instances
!pip install sagemaker-experiments tqdm

As well as the typical external libraries, we've provided a **set of prototype tools in the [/util](util) folder** sketching out some ideas for common cross-project utility code that can help simplify the workflow for regular use.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
import os

# External Dependencies:
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.pytorch.model import PyTorchModel
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

# Local Dependencies:
import util

Next we'll set some configurations, bearing in mind we are working in a data science sandbox within a broader (perhaps multi-user) project:

In [ ]:
role = sagemaker.get_execution_role()
smclient = boto3.client("sagemaker")
smsess = sagemaker.session.Session()


bucket_name = # TODO: Personal sandbox bucket
%store bucket_name

project_id = # TODO: Project ID from CloudFormation stack
%store project_id


bucket = boto3.resource("s3").Bucket(bucket_name)

project_config = util.project.init(project_id)  # Read project stack parameters from the AWS SSM store
project_config

## Create (or load) the **Experiment** in which we'll track our work:

An **"Experiment"** is a **collection of comparable "Trials"**.

Trials are more abstract and flexible than just training jobs, so the scope at which we define an "Experiment" will be determined by what we're trying to track and the capabilities of the Experiments functionality.

In this context we'll be comparing different ways to solve our classification problem, and have taken the decision (arguably driven by this being a code sample not an actual project) that this experiment be repeatable over time - so have appended a timestamp:

In [ ]:
experiment_name = util.append_timestamp("forest-cover-type")
experiment = util.smexps.create_or_load_experiment(
    experiment_name=experiment_name,
    description="Classification of forest type from cartographic variables",
    sagemaker_boto_client=smclient,  # (Optional)
)
%store experiment_name
print(experiment)

**"Trial Components"** are **discrete sub-stages** (e.g. processing, or training jobs) with associated of a **"Trial"**.

Trial Components can be associated to multiple trials, and that's exactly what we'll do here to record our initial **data pre-processing**.

A **Tracker** is a utility to record data to a Trial Component, and can be used either:

- As a Python context manager (with the `with` statement) for more stable-codified workflows (such as in the [public SM Experiments sample](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-experiments/mnist-handwritten-digits-classification-experiment.ipynb))
- ...Or just as an interactive interface as we will here

In [ ]:
preproc_tracker = Tracker.create(
    display_name="Preprocessing",
    sagemaker_boto_client=smclient,  # (Optional)
)
# (can also Tracker.load() from an existing Trial Component)

preproc_trial_component_name = preproc_tracker.trial_component.trial_component_name
%store preproc_trial_component_name

print(preproc_tracker.trial_component)

For the sake of illustration we could create a "Trial" including **only** the pre-processing step... But that would be abusing our intention to use the Experiment to record comparable tests!

In [ ]:
# preproc_trial = Trial.create(
#     trial_name=util.append_timestamp("preproc-only"), 
#     experiment_name=experiment.experiment_name,
#     sagemaker_boto_client=smclient,
# )
# preproc_trial.add_trial_component(preproc_trial_component)

## Download and Explore the Data

It's important to notice **we're sort of working backwards here**: The project environment has been provisioned, but the source data and target/test cases aren't loaded into it yet!

How stable are your raw source data and target variable definitions over the lifetime of an ML project? In practice we can expect some balance between:
- Working from target dataset+outcomes to a model
- Refining input features and problem definition from experimental findings

Here we'll show a pre-processing workflow with data logging to SageMaker Experiments, and then upload the final datasets to the project's common "source" bucket.

In [ ]:
raw_data_uri = "https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz"

!mkdir -p data/raw
!wget -O data/raw/covtype.data.gz $raw_data_uri
!wget -O data/raw/covtype.info https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.info
!gunzip -f data/raw/covtype.data.gz
assert os.path.isfile("data/raw/covtype.data")  # (Some of the shell cmds can fail without raising error)

preproc_tracker.log_input(
    name="UCI-Covertype",
    media_type="text/csv",
    value=raw_data_uri,
)

The data format is documented in the [data/raw/covtype.info](data/raw/covtype.info) we just downloaded

In [ ]:
cover_types = ("N/A", "Spruce/Fir", "Lodgepole Pine", "Ponderosa Pine", "Cottonwood/Willow", "Aspen", "Douglas-fir", "Krummholz")
wilderness_areas = ("Rawah", "Neota", "Comanche Peak", "Cache la Poudre")
preproc_tracker.log_parameters({
    "cover_types": cover_types,
    "n_cover_types": len(cover_types) - 1,
    "wilderness_areas": wilderness_areas,
    "n_wilderness_areas": len(wilderness_areas),
})

df_raw = pd.read_csv(
    "data/raw/covtype.data",
    names=[
        "Elevation_m",  # Elevation in meters
        "Aspect_deg",  # Aspect in degrees azimuth
        "Slope_deg",  # Slope in degrees
        "Horizontal_Distance_To_Hydrology_m",  # Horz Dist to nearest surface water features
        "Vertical_Distance_To_Hydrology_m",  # Vert Dist to nearest surface water features
        "Horizontal_Distance_To_Roadways_m",  # Horz Dist to nearest roadway
        "Hillshade_9am_uint8",  # Hillshade index at 9am, summer solstice
        "Hillshade_Noon_uint8",  # Hillshade index at noon, summer soltice
        "Hillshade_3pm_uint8",  # Hillshade index at 3pm, summer solstice
        "Horizontal_Distance_To_Fire_Points_m",  # Horz Dist to nearest wildfire ignition points
    ]
    + ["Area_is_{}".format(area.replace(" ", "_")) for area in wilderness_areas]
    + ["Soil_Type_is_{:02}".format(typ) for typ in range(1, 41)]
    + [
        "Cover_Type",  # Forest Cover Type designation
    ]
)

bool_columns = [col for col in df_raw.columns if "_is_" in col]

print(f"Raw dataframe shape (rows, cols) = {df_raw.shape}")
df_raw.head()

## Pre-Process and Split

This data is already quite well-conditioned, so all we really need to do is split training, validation and test sets:

In [ ]:
# This cell from the PyTorch-Tabnet Forest CoverType demo notebook is not actually necessary because the data is
# already clean:
# https://github.com/dreamquark-ai/tabnet/blob/develop/forest_example.ipynb

# from sklearn.preprocessing import LabelEncoder

# categorical_columns = []
# categorical_dims =  {}
# for col in df_raw.columns[df_raw.dtypes == object]:
#     print(col, df_raw[col].nunique())
#     l_enc = LabelEncoder()
#     # df_raw[col] = df_raw[col].fillna("Unknown")
#     df_raw[col] = l_enc.fit_transform(df_raw[col].values)
#     categorical_columns.append(col)
#     categorical_dims[col] = len(l_enc.classes_)

# preproc_tracker.log_parameters({
#     "categorical_columns": categorical_columns,
#     "normalization_std": categorical_dims,
# })

df_all = df_raw  # No pre-processing to do, so we'll just map same ref to a different variable name

In [ ]:
train_pct = 0.8
val_pct = 0.1
test_pct = 1. - train_pct - val_pct
preproc_tracker.log_parameters({
    "train_pct": train_pct,
    "val_pct": val_pct,
    "test_pct": test_pct,
})

df_train, df_val, df_test = np.split(
    df_all.sample(frac=1),
    [int(train_pct*len(df_all)), int((train_pct + val_pct)*len(df_all))]
)

print(f"Split randomly into train={len(df_train):,}, validation={len(df_val):,}, test={len(df_test):,} samples")

## Extract a Deliberately Biased Testing Subset

We'll extract a deliberately biased subset of our testing data to use later in demonstrating how SageMaker's **Model Monitoring** functionality can be applied to detect [Concept Drift](https://en.wikipedia.org/wiki/Concept_drift) over time after a live model endpoint is deployed.

Specifically we'll use the top X% of test records by `Elevation_m`: which is known to strongly affect the target variable.

In [ ]:
biascheck_field = "Elevation_m"
biascheck_test_pct = .2
preproc_tracker.log_parameters({
    "biascheck_field": biascheck_field,
    "biascheck_test_pct": biascheck_test_pct,
})

df_test_bias, _ = np.split(
    df_test.sort_values(biascheck_field, ascending=False),
    [int(biascheck_test_pct*len(df_test))]
)
# ...And re-randomize:
df_test_bias = df_test_bias.sample(frac=1)

# Simpler version of summary showing elevation only:
# pd.DataFrame({
#     "Test Set Elevation_m": df_test["Elevation_m"].describe(),
#     "Biased Subset Elevation_m": df_test_hielev["Elevation_m"].describe(),
# })

# Create summaries:
test_summary = df_test.describe()
test_bias_summary = df_test_bias.describe()

# Log summary metrics to Experiment:
for (dsname, summary) in (("test", test_summary), ("biastest", test_bias_summary)):
    for (fname, field) in (("feature", biascheck_field), ("target", "Cover_Type")):
        for stat in ("mean", "std"):
            preproc_tracker.log_metric(
                f"biascheck-{fname}-{dsname}-{stat}",
                summary[field][stat]
            )

# Present nested-column summary tables in the notebook:
test_summ_cpy = test_summary.copy()
test_summ_cpy.columns = pd.MultiIndex.from_product([["Test Set"], test_summ_cpy.columns])
test_bias_summ_cpy = test_bias_summary.copy()
test_bias_summ_cpy.columns = pd.MultiIndex.from_product([["Biased Subset"], test_bias_summ_cpy.columns])
pd.concat([test_summ_cpy, test_bias_summ_cpy], axis=1).loc[:, pd.IndexSlice[:, [biascheck_field, "Cover_Type"]]]

## Upload Prepared Datasets to S3

In [ ]:
# It's helpful to have our training and validation datasets with headers so we can reference columns by name
# in training hyperparams:
df_train.to_csv("data/train-withheader.csv", index=False)
df_val.to_csv("data/validation-withheader.csv", index=False)

# ...But useful to skip headers in our test datasets so we can push the files through batch transformations:
df_test.to_csv("data/test-noheader.csv", index=False, header=False)
df_test_bias.to_csv("data/test-bias-noheader.csv", index=False, header=False)

In [ ]:
# Let's save our columns list as well, since some files don't include them:
with open("data/columns.json", "w") as f:
    json.dump(df_train.columns.to_list(), f)

In [ ]:
# The upload() function returns the created S3 URI, allowing for nice feed-in to logging:
preproc_tracker.log_output(
    "columns",
    sagemaker.s3.S3Uploader.upload("data/columns.json", f"s3://{bucket_name}/data"),
    "application/json"
)
preproc_tracker.log_output(
    "train-csv",
    sagemaker.s3.S3Uploader.upload("data/train-withheader.csv", f"s3://{bucket_name}/data"),
    "text/csv"
)
preproc_tracker.log_output(
    "validation-csv",
    sagemaker.s3.S3Uploader.upload("data/validation-withheader.csv", f"s3://{bucket_name}/data"),
    "text/csv"
)
preproc_tracker.log_output(
    "test-csv",
    sagemaker.s3.S3Uploader.upload("data/test-noheader.csv", f"s3://{bucket_name}/data"),
    "text/csv"
)
preproc_tracker.log_output(
    "test-biased-csv",
    sagemaker.s3.S3Uploader.upload("data/test-bias-noheader.csv", f"s3://{bucket_name}/data"),
    "text/csv"
)

In [ ]:
# Double-check everything saves OK, because otherwise errors can be silent:
preproc_tracker.trial_component.save()

# Shouldn't need to do these as well, but could:
#preproc_trial.save()
#experiment.save()

## Promoting our problem definition from sandbox to project

This division and format of training, validation, and test data will be our current benchmark for the project.

We can use IAM roles to control access between individual sandbox and project-shared resources; and even provide separate-but-assumable roles for a "sudo"-style safety mechanism:

In [ ]:
sudosess = util.boto.assumed_role_session(
    project_config["SudoRole"],
)

In [ ]:
sudosess.resource("s3").Bucket(project_config["SourceBucket"]).upload_file(
    "data/test-noheader.csv",
    "data/test-noheader.csv",
)